<a href="https://colab.research.google.com/github/22053604/Cardiovascular-Health-Analysis/blob/main/Development_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Version Control**

# Pseudocode

....

In [1]:
#import necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [15]:
#Established a connection to the SQLite database
db_path = '/content/cardio_health.db'
conn = sqlite3.connect('cardiohealth.db')
cursor = conn.cursor()

#Retrieve and display table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

Tables in the database: [('medical_examination',)]


In [36]:
#Export all tables to CSV files &  Panadas dataframe
for table_name in tables:
    df = pd.read_sql(f"SELECT * FROM {table_name[0]}", conn)
    csv_file_name = f"{table_name[0]}.csv"
    df.to_csv(csv_file_name, index=False)
    print(f"Exported {table_name[0]} to {csv_file_name}")

#Load data from medical_examnination table
query = "SELECT * FROM medical_examination"
df = pd.read_sql(query, conn)

#Display tables
df = pd.read_csv('/content/medical_examination.csv')
print(df.head())

Exported medical_examination to medical_examination.csv
   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  
0     0       1       0  
1     0       1       1  
2     0       0       1  
3     0       1       1  
4     0       0       0  


In [44]:
#Show the missing values
print("Missing values in dataset: ")
print(df.isnull().sum())

# Remove duplicates and handle missing values
df_cleaned = df.drop_duplicates()

#Print table again after removing duplicates
print("After removing duplicates:")
print(df_cleaned.head())

#Converting age from days to years
df['age_years'] = df['age'] / 365



Missing values in dataset: 
id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
age_years      0
dtype: int64
After removing duplicates:
   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  age_years  
0     0       1       0  50.391781  
1     0       1       1  55.419178  
2     0       0       1  51.663014  
3     0       1       1  48.282192  
4     0       0       0  47.

In [25]:
#Grouping into category groups

#Define Age into bins and labels
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['0-9', '10-19','20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-100']
#Create age columns
df['age_group'] = pd.cut(df['age_years'], bins=age_bins, labels=age_labels, right=False)

#Transform Gender codes into understandable format (1 = female, 2 = male)
df['gender'] = df['gender'].map({1: 'female', 2: 'male'})

#Transform Cholesterol levels into readable categories
df['cholesterol'] = df['cholesterol'].map({1: 'Normal', 2: 'Above Normal', 3: 'Well Above Normal'})

In [26]:
# Data analysis functions
def analyse_combination(df, factors):
    query_str = " & ".join([f"{key} {value}" for key, value in factors.items()])
    filtered_data = df.query(query_str)
    return filtered_data

# Example analysis: Demographic and Lifestyle Factors
factors_combination_1 = {'gender': "== 'female'", 'smoke': '== 1'}
combination_1_result = analyse_combination(df, factors_combination_1)
print("Female smokers:")
print(combination_1_result.head())

Female smokers:
Empty DataFrame
Columns: [id, age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active, cardio, age_years, age_group]
Index: []


In [27]:
gender_smoke_data = df.groupby(['gender','smoke'])['cardio'].mean().unstack()
gender_smoke_data.plot(kind='bar', stacked=True)
plt.title('Cardiovascular Risk by Gender and Smoking Status')
plt.xlabel('Gender')
plt.ylabel('Cardiovascular Risk')
plt.show()

TypeError: no numeric data to plot

Note: 0 blue & 1 orange = should be smoker/non-smoker
Risk in percentage
